Libraries

In [4]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv("C:/Users/hp/Documents/Countries Worldwide Info/.env")

True

mysql info

In [5]:
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_NAME = os.getenv("DB_NAME")
DB_HOST = "127.0.0.1:3307"

Load the clean data to Mysql

In [6]:
def load_to_mysql(df):
    try:
        print("Starting load_to_mysql...")
        df.to_csv(r"C:\Users\hp\Documents\Countries Worldwide Info\cleaned_countriesdata.csv", index=False)
        print("CSV saved successfully.")
        conn = mysql.connector.connect(
            host="127.0.0.1",
            port=3307,
            user=DB_USER,
            password=DB_PASS,
            database=DB_NAME
        )
        cursor = conn.cursor()
        print("MySQL connector connection successful.")

        # Drop table if exists
        cursor.execute("DROP TABLE IF EXISTS cleancountries")

        # Create table
        create_table_query = """
        CREATE TABLE cleancountries (
            Country VARCHAR(255),
            Continent VARCHAR(255),
            Area FLOAT,
            Population BIGINT,
            `Capital City` VARCHAR(255)
        )
        """
        cursor.execute(create_table_query)
        print("Table cleancountries created.")

        # Insert data
        insert_query = """
        INSERT INTO cleancountries (Country, Continent, Area, Population, `Capital City`)
        VALUES (%s, %s, %s, %s, %s)
        """
        for _, row in df.iterrows():
            values = (
                row['Country'],
                row['Continent'],
                row['Area'],
                row['Population'],
                row['Capital City'] if pd.notnull(row['Capital City']) else None
            )
            cursor.execute(insert_query, values)
        conn.commit()
        print("✅ Data successfully uploaded to MySQL.")
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error loading data to MySQL: {e}")
        raise